In [3]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import time
from threading import Thread
import yaml


In [9]:
fp = 'park-grass-10-18-2024_rgn-ortho.tif'
out = 'NDVI_rasterio.tif'
src = rasterio.open(fp)
array = src.read

In [13]:
def calc_tif_ndvi(fp, output_path='ndvi_output.tif'):
    # Function to print progress in the background
    print("NDVI processing underway.")
    def print_progress():
        start_time = time.time()
        last_report = 0
        while not processing_done:
            elapsed = time.time() - start_time
            if elapsed >= last_report + 10:
                print(f"Processing... {int(elapsed//10)*10} seconds elapsed")
                last_report = elapsed//10 * 10
            time.sleep(0.1)  # Check more frequently
    
    processing_done = False
    # Start the progress thread
    progress_thread = Thread(target=print_progress)
    progress_thread.start()

    try:
        with rasterio.open(fp) as src:
            # Read bands and calculate NDVI
            red = src.read(1).astype(float) / 10000  # Band 1 = Red
            nir = src.read(3).astype(float) / 10000  # Band 3 = NIR

            # Calculate NDVI
            with np.errstate(divide='ignore', invalid='ignore'):
                ndvi = (nir - red) / (nir + red)
                ndvi = np.where((nir + red) == 0, 0, ndvi)  # Handle division by zero
                ndvi = np.clip(ndvi, -1, 1)  # Enforce valid range

            # Get metadata from source file
            profile = src.profile
            profile.update(
                dtype=rasterio.float32,
                count=1,
                nodata=np.nan,
                driver='GTiff'
            )

            # Save the NDVI raster
            with rasterio.open(output_path, 'w', **profile) as dst:
                dst.write(ndvi.astype(rasterio.float32), 1)
                dst.set_band_description(1, 'NDVI')

        print("NDVI range:", np.nanmin(ndvi), np.nanmax(ndvi))  # Should be [-1, 1]
        print(f"NDVI processing finished! Result saved to {output_path}")
    
    finally:
        processing_done = True
        progress_thread.join()  # Wait for the progress thread to finish

In [12]:
calc_tif_ndvi(fp, out)

Processing... 10 seconds elapsed
Processing... 20 seconds elapsed
Processing... 30 seconds elapsed
Processing... 40 seconds elapsed
Processing... 50 seconds elapsed
Processing... 60 seconds elapsed
Processing... 70 seconds elapsed
Processing... 80 seconds elapsed
Processing... 90 seconds elapsed
Processing... 100 seconds elapsed
Processing... 110 seconds elapsed
Processing... 120 seconds elapsed
Processing... 130 seconds elapsed
Processing... 140 seconds elapsed
Processing... 150 seconds elapsed
Processing... 160 seconds elapsed
NDVI range: -1.0 0.9012812752219532
NDVI processing finished! Result saved to NDVI_rasterio.tif
